# Exercise Set 13: Network formation


In this Exercise Set 13 we will investigate network formation among high school pupils. 

## Part 1: Network formation


Load the data using the script below. Read a bit about the dataset [here](http://www.sociopatterns.org/datasets/high-school-contact-and-friendship-networks/) to get an understanding of what is in each variable. 

The script gives you two dataframes to work with: 
 > `el`, which is an edge-list 
 >
 > `ind` which contains individual characteristics

In [46]:
import networkx as nx
import numpy as np
import pandas as pd
from datetime import datetime

url_base = 'http://www.sociopatterns.org/wp-content/uploads/2015/'

# edgelist
url_el = url_base + '07/High-School_data_2013.csv.gz'
col_names_el = ['timestamp', 'u1', 'u2', 'class1', 'class2']
el = pd.read_csv(url_el, header=None, names=col_names_el, delimiter=' ')

# individual characteristics
url_ind = url_base + '09/metadata_2013.txt'
col_names_ind = ['u', 'class', 'gender']
ind = pd.read_csv(url_ind, header=None, names=col_names_ind, delimiter='\t')\
            .set_index('u')

# remove observation with missing gender
has_gender = ind[ind.gender!='Unknown'].index

# DataFrames
ind = ind.loc[has_gender].copy()
el = el[el.u1.isin(has_gender) &  el.u2.isin(has_gender)].copy()

## Add hour col
el = el\
.assign(ts_date = lambda df:pd.to_datetime(df.timestamp,unit='s').dt.strftime('%Y-%m-%d %H:00:00'))

el.head(5)

,timestamp,u1,u2,class1,class2,ts_date
0,1385982020,454,640,MP,MP,2013-12-02 11:00:00
1,1385982020,1,939,2BIO3,2BIO3,2013-12-02 11:00:00
2,1385982020,185,258,PC*,PC*,2013-12-02 11:00:00
3,1385982020,55,170,2BIO3,2BIO3,2013-12-02 11:00:00
4,1385982020,9,453,PC,PC,2013-12-02 11:00:00


> **Ex. 13.1.1**: Describe the edgelist columns content. Parse the timestamp. What is the resolution of meetings? Use the parsed timestamp to count the meetings by hour in local time.

### Answer to Ex 13.1.1:
1. *timestamp is a unix format for the occurence of a meeting between two students. The timestamps are divided into intervals of 20 seconds. Hence, it records if two students had a meeting during a specific 20 second interval.*

2. *u1 is the student id of the first student in the meeting*

3. *u2 is the student id of the second student in the meeting*

4. *class1 is the class id of the first student in the meeting*

5. *class2 is the class id of the second studeent in the meeting*

6. *ts_date is timedate variable showing the date within hour*

In [48]:
print("# of meetings by hour")
el.groupby('ts_date')['timestamp'].count().to_frame().rename(columns={'timestamp':'n_meetings'})

# of meetings by hour


,n_meetings
ts_date,
2013-12-02 11:00:00,5556
2013-12-02 12:00:00,4259
2013-12-02 13:00:00,6617
2013-12-02 14:00:00,5715
2013-12-02 15:00:00,5972
2013-12-03 07:00:00,6048
2013-12-03 08:00:00,5286
2013-12-03 09:00:00,7104
2013-12-03 10:00:00,5096


> **Ex. 13.1.2**: Count the number of meetings for each edge and save this as a DataFrame called `el_agg`. Filter out edges with less than 5 minutes of meetings. Attach the gender and class of both nodes.

In [87]:
el_agg = el\
.groupby(['u1','u2'])['timestamp']\
.count()\
.to_frame()\
.reset_index()\
.rename(columns={'timestamp':'n_meetings','u1':'u'})\
.query('n_meetings>=5')\
.merge(ind.reset_index(),on='u',how='left')\
.rename(columns={'u':'u1','class':'class1','gender':'gender1','u2':'u'})\
.merge(ind.reset_index(),on='u',how='left')\
.rename(columns={'u':'u2','class':'class2','gender':'gender2'})

el_agg.head(5)

,u1,u2,n_meetings,class1,gender1,class2,gender2
0,1,55,8,2BIO3,M,2BIO3,F
1,1,117,18,2BIO3,M,2BIO3,M
2,1,170,8,2BIO3,M,2BIO3,F
3,1,196,38,2BIO3,M,2BIO3,M
4,1,205,47,2BIO3,M,2BIO3,M


> **Ex. 13.1.3**: Answer question in the function `fraction_triangles` below. Explain how `fraction_triangles` is related to  computing the clustering coefficient (using `nx.average_clustering`).
>
>> *Hint:* The following code does the same thing as `fraction_triangles`, but at a scale where you can understand what's going on. If you have a hard time understanding the code in the function you can try to play around with this simpler example
>>
>> ```python
>> import networkx as nx 
>>
>> A  = np.array(
>>     [[0, 1, 1, 0],
>>      [1, 0, 1, 0],
>>      [1, 1, 0, 1],
>>      [0, 0, 1, 0]]
>> )
>>
>> G = nx.from_numpy_array(A)
>> nx.draw(G,with_labels=True)
>>
>> def nth(A, n):
>>     A_ = A.copy()    
>>     for _ in range(1,n):
>>         A = A.dot(A_)
>>     return A
>>
>> a_t = nth(A,3).diagonal().sum()/6
>> n = len(A[:,0])
>> p_t = binom(n, 3)
>> ```


### Answer to Ex 13.1.3:
*The questions seem to be answered already?*

In [99]:
def make_net(el_, nodes):
    '''
    Convert edgelist to networkx graph which is 
    binary and undirected.
    
    Parameters
    ----------
    el_ : DataFrame
        Table containing an edgelist with columns 
        `u1` and `u2` which are the nodes in the edge.
        
    nodes : array-like
        1d array containing the node identities.
    '''    
    
    nx_input = el_, 'u1', 'u2', 'n_meetings', nx.Graph()
    g = nx.from_pandas_edgelist(*nx_input)
    g.add_nodes_from(nodes)
    return g

In [100]:
from scipy.special import binom

def fraction_triangles(el_, nodes):
    '''
    Compute fraction of actual triangles out 
    of the potential triangles.
    
    Parameters
    ----------
    el_ : DataFrame
        Table containing an edgelist with columns 
        `u1` and `u2` which are the nodes in the edge.
        
    nodes : array-like
        1d array containing the node identities.
    '''
    
    g = make_net(el_, nodes)
    
    #Q.1: what is `A`?: the adjacency matrix which is symmetric and binary
    #Q.2: what does `A**3` do? compute the number of paths between two nodes
    #Q.3: what is diagonal of A_t? the number of actual paths of length 3, 
    # i.e. triangles, which include the person. these are called cycles
    # because they start and end at the same person
    
    # count actual triangles    
    A = nx.to_scipy_sparse_matrix(g)
    A_t = A**3
    a_t = A_t.diagonal().sum()/6
    
    #Q.4: what does `binom(n,3)` compute? the number of triangles including the person
    
    # count potential triangles
    n = len(g.nodes())
    p_t = binom(n, 3)
        
    return a_t/p_t

> **Ex. 13.1.4**: Apply the function `fraction_triangles` to `el_agg` and print the triangle fraction in the network. Next remove all edges that go between classes. Compute triangle fraction within each class and store it. Compute the mean within class triangles and bootstrap the standard error of the mean. Comment on the output.
>
>> *Hint:* To bootstrap an estimate draw $k>>0$ samples with replacement from the data. Compute the estimate on each of these samples and average them in the end to get the bootstrapped estimate. 

In [167]:
## ANSWER TO EX 13.1.4

import random

el_agg_within_class = el_agg.query('class1==class2')

def nodes_as_list(df,col_names=['u1','u1']):
    """
    returns a list of all unique nodes from a df with nodes
    """
    nodes = np.unique(df.u1.unique().tolist()+df.u2.unique().tolist()).tolist()
    return nodes

def mean_triangles_within_class(df):
    """
    calculates the mean fraction of triangles within classes
    """
    mean_triangles_within_class = df\
    .groupby('class1')\
    .apply(lambda df_g:fraction_triangles(el_=df_g,nodes=nodes_as_list(df_g)))\
    .mean()
    return mean_triangles_within_class

def bootstrap_df(df,k):
    """
    creates a boostrapped dataframe.
    
    arguments:
    
    df: pandas dataframe
    - should contain edges and class1
    
    k: int
    - number of draws (with replacements)
    
    """
    df      = df.reset_index(drop=True).copy()
    indices = random.choices(df.reset_index(drop=True).index,k=k)
    df      = df.loc[df.index.isin(indices),:]
    return df

def std_of_mean(df,bootstraps,k):
    """
    calculates the standard error of mean fraction of triangles within class on bootstrapped dataframes
    
    arguments:

    df: pandas dataframe
    - should contain edges and class1
    
    bootstraps: int
    - number of bootstraps
    
    k: int
    - number of draws (with replacements)    
    
    """
    
    mean_frq_tri_list = []
    for _ in range(bootstraps):
        df_bootstrap = bootstrap_df(df,k=k)
        mean_frq_tri_list.append(mean_triangles_within_class(df_bootstrap))
    return np.std(mean_frq_tri_list)

# List of unique nodes
nodes_all           = nodes_as_list(el_agg)
nodes_within_class  = nodes_as_list(el_agg_within_class)

# Compute fraction of triangles
f_triangles_all                 = fraction_triangles(el_=el_agg,nodes=nodes)
mean_freq_tri_within_class      = mean_triangles_within_class(el_agg_within_class)
std_mean_freq_tri_within_class  = std_of_mean(el_agg_within_class,bootstraps=100,k=500)

print(f'Fraction of triangles for full network: {f_triangles_all}\n Mean fraction of triagles for network within classes: {mean_freq_tri_within_class}\n Bootstrapped std is: {std_mean_freq_tri_within_class}')


Fraction of triangles for full network: 0.0012187660870313175
 Mean fraction of triagles for network within classes: 0.08729511149957794
 Bootstrapped std is: 0.00019288130539339972


### Answer to ex 13.1.4:
*The within class has a higher frequency of triangles. This is not surprising as we would expect students to form stronger sub networks within classes. We would expect this, because the students within the same classes are exposed the so-called spatial network formation explanation. That is, the students are frequently in the same time and space dimension, increasing the likelihood of them to form networks.*

Recall from class that we can define the following measures of homophily. We define **homophily index** inspired by [Currarini et al. (2009)](https://doi.org/10.2139/ssrn.1021650):
- share of edges that are same type: $H = \frac{s}{s+d}$
- possible range [0,1]


We define **baseline homophily** as: 
- We count fraction of potential edges in population of nodes which are same type:

\begin{equation}B=\frac{\sum_t\#potential(n_t)}{\#potential(n)}, \qquad \#potential(k)=\frac{k\cdot(k-1)}{2}\end{equation}

- Interpretation: Expected homophily from random link formation.     

We define **inbreeding homophily** as:      

\begin{equation}IH=\frac{H-B}{1-B}\end{equation}


> **Ex. 13.1.5**: Compute the inbreeding homophily for each class. Use the class measures to compute the mean. Use a bootstrap to compute whether there is inbreeding homophily.

In [208]:
el_agg = el_agg\
.assign(same_gender = lambda df:(df.gender1==df.gender2))

def homophily(df):
    """
    Calculates homophily from formula H = s/s+d
    
    Arguments:
    df: pandas dataframe
    - should contain the edges and a bool column 'same_gender'.
    
    returns:
    H: float
    - homophily in [0,1] range    
    """
    
    H = df.same_gender.sum()/df.shape[0]
    return H

def baseline_homophily(ind_df,nodes):
    """
    Calculates baseline homophily from formula B = sum(potential(n_t))/potential(n)
    
    Arguments:
    ind_df: pandas dataframe
    - should contain gender info of each node.
    
    nodes: list
    - list with unique nodes in network
    
    returns:
    B: float
    - baseline homophily in [0,1] range    
    """
    ind_df = ind_df.reset_index()
    ind_df = ind_df\
    .loc[ind_df['u'].isin(nodes),:]
    
    k_list      = ind_df.gender.value_counts()
    n           = k_list.shape[0]
    k_potential = [k*(k-1) for k in k_list]
    k_potential = np.sum(k_potential)/n
    n_potential = (len(nodes)*(len(nodes)-1))/2
    B           = k_potential/n_potential
    return B

def inbreeding_homophily(df,ind_df):
    """
    Calculates inbreeding homophily from formula IH = (H - B)/(1 - B) 
    
    Arguments:
    df: pandas dataframe
    - contains the nodes
    
    ind_df: pandas dataframe
    - should contain gender info of each node.    
    
    returns:
    IH: float
    - imbreeding homophily in [0,1] range    
    """
    H     = homophily(df)
    nodes = nodes_as_list(df)
    B     = baseline_homophily(ind_df,nodes)
    IH    = (H-B)/(1-B)
    return IH

def std_of_mean_I_H(df,ind_df,bootstraps,k):
    """
    Calculates standard error of mean imbreeding homophily within classes. 
    
    Arguments:
    df: pandas dataframe
    - contains the nodes
    
    ind_df: pandas dataframe
    - should contain gender info of each node.    
    
    bootstraps: int
    - number of bootstraps
    
    k:int
    - number of draws within each bootstrap (with replacements) 
       
    """
    result_list = []
    for _ in range(bootstraps):
        df_bootstrap = bootstrap_df(df,k=k)
        mean_I_h     = df_bootstrap\
        .groupby('class1')\
        .apply(lambda df_g:inbreeding_homophily(df_g,ind))\
        .mean()
        
        result_list.append(mean_I_h)
    return np.std(result_list)

In [211]:
mean_I_H = el_agg\
.groupby('class1')\
.apply(lambda df_g:inbreeding_homophily(df_g,ind))\
.mean()

std_I_H  = std_of_mean_I_H(el_agg,ind,bootstraps=100,k=500)

print(f"The mean inbreeding homophily within classes is: {mean_I_H} ({std_I_H})")

The mean inbreeding homophily within classes is: 0.10733391395607662 (0.05007491190505416)


### Answer to 13.1.5:
*The estimated mean imbreeding homophily is 0.107 with a standard error of 0.050. Hence, we may conclude that the students have a borderline positive-significant imbreeding homophily. That is, the students are more likely to link with a person of the same gender than what we would expect had the network formation been random.* 

> **Ex. 13.1.6** (BONUS): Describe what an unsupported edge is. Construct a test of whether there is a preference for forming  triangles within same gender than across.
>
>> *Hint:*  You can find inspiration in the approach of [Chandrasekhar, Jackson (2018)](https://web.stanford.edu/~arungc/CJ_sugm.pdf) pp. 31-35. They construct an almost identical test for triangle formation across castes in Indian villages.

In [ ]:
# [Answer to ex. 13.1.6 here]